In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Sales").getOrCreate()

df = spark.read.csv("Sales_Data.csv",header=True,inferSchema=True)

df.show()

24/12/30 00:05:20 WARN Utils: Your hostname, vinzzmacbook.local resolves to a loopback address: 127.0.0.1; using 192.168.0.200 instead (on interface en0)
24/12/30 00:05:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/30 00:05:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+---+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
|_c0|Order ID|             Product|Quantity Ordered|Price Each|      Order Date|    Purchase Address|Month| Sales|          City|Hour|
+---+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
|  0|  295665|  Macbook Pro Laptop|               1|    1700.0|30-12-2019 00:01|136 Church St, Ne...|   12|1700.0| New York City|   0|
|  1|  295666|  LG Washing Machine|               1|     600.0|29-12-2019 07:03|562 2nd St, New Y...|   12| 600.0| New York City|   7|
|  2|  295667|USB-C Charging Cable|               1|     11.95|12-12-2019 18:21|277 Main St, New ...|   12| 11.95| New York City|  18|
|  3|  295668|    27in FHD Monitor|               1|    149.99|22-12-2019 15:13|410 6th St, San F...|   12|149.99| San Francisco|  15|
|  4|  295669|USB-C Charging Cable|               1|   

24/12/30 00:05:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Order ID, Product, Quantity Ordered, Price Each, Order Date, Purchase Address, Month, Sales, City, Hour
 Schema: _c0, Order ID, Product, Quantity Ordered, Price Each, Order Date, Purchase Address, Month, Sales, City, Hour
Expected: _c0 but found: 
CSV file: file:///Users/vinzz/pyspark/Sales_Data.csv
24/12/30 00:05:36 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]).show()

24/12/30 00:06:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Order ID, Product, Quantity Ordered, Price Each, Order Date, Purchase Address, Month, Sales, City, Hour
 Schema: _c0, Order ID, Product, Quantity Ordered, Price Each, Order Date, Purchase Address, Month, Sales, City, Hour
Expected: _c0 but found: 
CSV file: file:///Users/vinzz/pyspark/Sales_Data.csv


+---+--------+-------+----------------+----------+----------+----------------+-----+-----+----+----+
|_c0|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|Month|Sales|City|Hour|
+---+--------+-------+----------------+----------+----------+----------------+-----+-----+----+----+
|  0|       0|      0|               0|         0|         0|               0|    0|    3|   0|   0|
+---+--------+-------+----------------+----------+----------+----------------+-----+-----+----+----+



In [4]:
df = df.fillna({"Sales": df.agg(avg("Sales")).collect()[0][0]})
# df = df.dropna(subset=["Sales","Quantity Ordered"])

In [7]:
df = df.dropDuplicates()

In [8]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Sales: double (nullable = false)
 |-- City: string (nullable = true)
 |-- Hour: integer (nullable = true)



In [9]:
df = df.withColumn("Quantity Ordered",col("Quantity Ordered").cast("int"))\
.withColumn("Price Each",col("Price Each").cast("double"))\
.withColumn("Sales",col("Sales").cast("double"))\

df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Sales: double (nullable = false)
 |-- City: string (nullable = true)
 |-- Hour: integer (nullable = true)



In [10]:
df = df.filter((col("Sales") >= 0) & (col("Quantity Ordered") >= 0) & (col("Price Each") >= 0))

In [12]:
df.groupBy("Product").sum("Sales").show()

24/12/30 00:13:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Order ID, Product, Quantity Ordered, Price Each, Order Date, Purchase Address, Month, Sales, City, Hour
 Schema: _c0, Order ID, Product, Quantity Ordered, Price Each, Order Date, Purchase Address, Month, Sales, City, Hour
Expected: _c0 but found: 
CSV file: file:///Users/vinzz/pyspark/Sales_Data.csv
[Stage 13:=========>                                                (1 + 5) / 6]

+--------------------+------------------+
|             Product|        sum(Sales)|
+--------------------+------------------+
|    Wired Headphones|246651.93376000616|
|  Macbook Pro Laptop|         8037600.0|
|Apple Airpods Hea...|         2349150.0|
|              iPhone|         4794300.0|
|Lightning Chargin...|347094.14999999467|
|Bose SoundSport H...| 1345565.429999996|
|USB-C Charging Cable|286674.79376000166|
|AAA Batteries (4-...|   92740.829999996|
|        20in Monitor| 454148.7099999983|
|    27in FHD Monitor|1132424.5000000102|
|     Vareebadd Phone|          827200.0|
|34in Ultrawide Mo...|2355558.0099999746|
|            LG Dryer|          387600.0|
|AA Batteries (4-p...|106300.05375999905|
|        Google Phone|         3319200.0|
|       Flatscreen TV|         1445700.0|
|  LG Washing Machine|          399600.0|
|27in 4K Gaming Mo...|2435097.5599999735|
|     ThinkPad Laptop|4129958.6999999736|
+--------------------+------------------+



In [14]:
df.coalesce(1).write.csv("sales_prac",header=True)

24/12/30 00:14:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Order ID, Product, Quantity Ordered, Price Each, Order Date, Purchase Address, Month, Sales, City, Hour
 Schema: _c0, Order ID, Product, Quantity Ordered, Price Each, Order Date, Purchase Address, Month, Sales, City, Hour
Expected: _c0 but found: 
CSV file: file:///Users/vinzz/pyspark/Sales_Data.csv
                                                                                